In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot 
import keras #importing keras using tensorflow as backend

from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from sklearn.model_selection  import  train_test_split

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM

In [3]:
df=pd.read_csv('/content/drive/My Drive/bitcoin_ticker.csv')

In [4]:
df

,date_id,datetime_id,market,rpt_key,last,diff_24h,diff_per_24h,bid,ask,low,high,volume,created_at,updated_at
0,2017-05-31,2017-06-01 00:00:00,bitstamp,btc_eur,1.996720e+03,2.029990e+03,-1.638924,2.005500e+03,2.005560e+03,1.950000e+03,2.063730e+03,2314.500750,2017-05-31 14:59:36,2017-05-31 14:59:36
1,2017-05-31,2017-06-01 00:00:00,bitflyer,btc_jpy,2.670980e+05,2.696490e+05,-0.946045,2.671240e+05,2.672670e+05,2.671240e+05,2.672670e+05,70922.880112,2017-05-31 14:59:36,2017-05-31 14:59:36
2,2017-05-31,2017-06-01 00:00:00,korbit,btc_krw,3.003500e+06,3.140000e+06,-4.347134,3.003500e+06,3.004000e+06,3.002000e+06,3.209500e+06,6109.752872,2017-05-31 14:59:36,2017-05-31 14:59:36
3,2017-05-31,2017-06-01 00:00:00,bitstamp,btc_usd,2.237400e+03,2.239370e+03,-0.087971,2.233090e+03,2.237400e+03,2.154280e+03,2.293460e+03,13681.282017,2017-05-31 14:59:36,2017-05-31 14:59:36
4,2017-05-31,2017-06-01 00:00:00,okcoin,btc_usd,2.318820e+03,2.228700e+03,4.043613,2.319400e+03,2.319990e+03,2.129780e+03,2.318820e+03,4241.641516,2017-05-31 14:59:36,2017-05-31 14:59:36
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
627185,2017-07-14,2017-07-14 13:18:00,korbit,etc_krw,2.090000e+04,2.331000e+04,-10.338910,2.084000e+04,2.091000e+04,2.000000e+04,2.340000e+04,842321.282598,2017-07-14 04:17:20,2017-07-14 04:17:20
627186,2017-07-14,2017-07-14 13:18:00,bitflyer,eth_btc,8.630000e-02,9.410000e-02,-8.289054,8.620000e-02,8.675000e-02,8.620000e-02,8.675000e-02,4448.239195,2017-07-14 04:17:20,2017-07-14 04:17:20
627187,2017-07-14,2017-07-14 13:18:00,korbit,eth_krw,2.391000e+05,2.689500e+05,-11.098717,2.386000e+05,2.391000e+05,2.350000e+05,2.690000e+05,117124.419358,2017-07-14 04:17:20,2017-07-14 04:17:20
627188,2017-07-14,2017-07-14 13:18:00,bitflyer,fx_btc_jpy,2.615530e+05,2.713100e+05,-3.596255,2.615540e+05,2.616260e+05,2.615540e+05,2.616260e+05,73814.151389,2017-07-14 04:17:20,2017-07-14 04:17:20


In [5]:
df.head(5)

,date_id,datetime_id,market,rpt_key,last,diff_24h,diff_per_24h,bid,ask,low,high,volume,created_at,updated_at
0,2017-05-31,2017-06-01 00:00:00,bitstamp,btc_eur,1996.72,2029.99,-1.638924,2005.50,2005.56,1950.00,2063.73,2314.500750,2017-05-31 14:59:36,2017-05-31 14:59:36
1,2017-05-31,2017-06-01 00:00:00,bitflyer,btc_jpy,267098.00,269649.00,-0.946045,267124.00,267267.00,267124.00,267267.00,70922.880112,2017-05-31 14:59:36,2017-05-31 14:59:36
2,2017-05-31,2017-06-01 00:00:00,korbit,btc_krw,3003500.00,3140000.00,-4.347134,3003500.00,3004000.00,3002000.00,3209500.00,6109.752872,2017-05-31 14:59:36,2017-05-31 14:59:36
3,2017-05-31,2017-06-01 00:00:00,bitstamp,btc_usd,2237.40,2239.37,-0.087971,2233.09,2237.40,2154.28,2293.46,13681.282017,2017-05-31 14:59:36,2017-05-31 14:59:36
4,2017-05-31,2017-06-01 00:00:00,okcoin,btc_usd,2318.82,2228.70,4.043613,2319.40,2319.99,2129.78,2318.82,4241.641516,2017-05-31 14:59:36,2017-05-31 14:59:36


In [6]:
df.tail()

,date_id,datetime_id,market,rpt_key,last,diff_24h,diff_per_24h,bid,ask,low,high,volume,created_at,updated_at
627185,2017-07-14,2017-07-14 13:18:00,korbit,etc_krw,20900.0000,23310.0000,-10.338910,20840.0000,20910.00000,20000.0000,23400.00000,842321.282598,2017-07-14 04:17:20,2017-07-14 04:17:20
627186,2017-07-14,2017-07-14 13:18:00,bitflyer,eth_btc,0.0863,0.0941,-8.289054,0.0862,0.08675,0.0862,0.08675,4448.239195,2017-07-14 04:17:20,2017-07-14 04:17:20
627187,2017-07-14,2017-07-14 13:18:00,korbit,eth_krw,239100.0000,268950.0000,-11.098717,238600.0000,239100.00000,235000.0000,269000.00000,117124.419358,2017-07-14 04:17:20,2017-07-14 04:17:20
627188,2017-07-14,2017-07-14 13:18:00,bitflyer,fx_btc_jpy,261553.0000,271310.0000,-3.596255,261554.0000,261626.00000,261554.0000,261626.00000,73814.151389,2017-07-14 04:17:20,2017-07-14 04:17:20
627189,2017-07-14,2017-07-14 13:18:00,okcoin,ltc_usd,45.9310,48.9170,-6.104217,45.9540,46.20000,45.2410,49.07500,52799.710000,2017-07-14 04:17:20,2017-07-14 04:17:20


##  Data Preprocessing

### The value_counts() function is used to get a Series containing counts of unique values.rpt_key consists of different kinds of currencies

In [7]:
df['rpt_key'].value_counts()

btc_usd       125438
fx_btc_jpy     62719
btc_jpy        62719
btc_eur        62719
ltc_usd        62719
etc_krw        62719
eth_krw        62719
btc_krw        62719
eth_btc        62719
Name: rpt_key, dtype: int64

### Price in various currencies are given-Considering only USD 

In [8]:
df= df.loc[(df['rpt_key']=='btc_usd')]

In [9]:
df

,date_id,datetime_id,market,rpt_key,last,diff_24h,diff_per_24h,bid,ask,low,high,volume,created_at,updated_at
3,2017-05-31,2017-06-01 00:00:00,bitstamp,btc_usd,2237.40,2239.37,-0.087971,2233.09,2237.40,2154.28,2293.46,13681.282017,2017-05-31 14:59:36,2017-05-31 14:59:36
4,2017-05-31,2017-06-01 00:00:00,okcoin,btc_usd,2318.82,2228.70,4.043613,2319.40,2319.99,2129.78,2318.82,4241.641516,2017-05-31 14:59:36,2017-05-31 14:59:36
15,2017-06-01,2017-06-01 00:01:00,bitstamp,btc_usd,2248.39,2242.44,0.265336,2247.77,2248.38,2154.28,2293.46,13701.698603,2017-05-31 15:00:36,2017-05-31 15:00:36
16,2017-06-01,2017-06-01 00:01:00,okcoin,btc_usd,2320.42,2228.40,4.129420,2320.99,2321.49,2129.78,2322.00,4260.261516,2017-05-31 15:00:36,2017-05-31 15:00:36
23,2017-06-01,2017-06-01 00:02:00,bitstamp,btc_usd,2248.35,2238.58,0.436437,2248.35,2248.69,2154.28,2293.46,13742.110913,2017-05-31 15:01:36,2017-05-31 15:01:36
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
627164,2017-07-14,2017-07-14 13:16:00,okcoin,btc_usd,2394.03,2500.01,-4.239183,2394.35,2405.72,2378.02,2529.20,1111.540000,2017-07-14 04:15:20,2017-07-14 04:15:20
627173,2017-07-14,2017-07-14 13:17:00,bitstamp,btc_usd,2320.47,2407.06,-3.597335,2320.48,2322.55,2307.46,2413.60,7969.263583,2017-07-14 04:16:20,2017-07-14 04:16:20
627174,2017-07-14,2017-07-14 13:17:00,okcoin,btc_usd,2394.03,2500.01,-4.239183,2394.42,2405.69,2378.02,2529.20,1111.540000,2017-07-14 04:16:20,2017-07-14 04:16:20
627183,2017-07-14,2017-07-14 13:18:00,bitstamp,btc_usd,2320.47,2408.00,-3.634967,2320.57,2322.55,2307.46,2413.60,7968.970715,2017-07-14 04:17:20,2017-07-14 04:17:20


In [10]:
df.head()

,date_id,datetime_id,market,rpt_key,last,diff_24h,diff_per_24h,bid,ask,low,high,volume,created_at,updated_at
3,2017-05-31,2017-06-01 00:00:00,bitstamp,btc_usd,2237.40,2239.37,-0.087971,2233.09,2237.40,2154.28,2293.46,13681.282017,2017-05-31 14:59:36,2017-05-31 14:59:36
4,2017-05-31,2017-06-01 00:00:00,okcoin,btc_usd,2318.82,2228.70,4.043613,2319.40,2319.99,2129.78,2318.82,4241.641516,2017-05-31 14:59:36,2017-05-31 14:59:36
15,2017-06-01,2017-06-01 00:01:00,bitstamp,btc_usd,2248.39,2242.44,0.265336,2247.77,2248.38,2154.28,2293.46,13701.698603,2017-05-31 15:00:36,2017-05-31 15:00:36
16,2017-06-01,2017-06-01 00:01:00,okcoin,btc_usd,2320.42,2228.40,4.129420,2320.99,2321.49,2129.78,2322.00,4260.261516,2017-05-31 15:00:36,2017-05-31 15:00:36
23,2017-06-01,2017-06-01 00:02:00,bitstamp,btc_usd,2248.35,2238.58,0.436437,2248.35,2248.69,2154.28,2293.46,13742.110913,2017-05-31 15:01:36,2017-05-31 15:01:36


### datetime_id to datatime

In [11]:
df1=df.reset_index(drop=True)['last']

In [12]:
df1

0         2237.40
1         2318.82
2         2248.39
3         2320.42
4         2248.35
           ...   
125433    2394.03
125434    2320.47
125435    2394.03
125436    2320.47
125437    2394.03
Name: last, Length: 125438, dtype: float64

### Feature scaling (Scaling  last values between 0-1)

In [13]:
scaler=MinMaxScaler(feature_range=(0,1))
df1=scaler.fit_transform(np.array(df1).reshape(-1,1))

In [14]:
print(df1)

[[0.        ]
 [0.08614141]
 [0.01162729]
 ...
 [0.16571271]
 [0.08788709]
 [0.16571271]]


### Splitting  dataset into training and testing

In [15]:
##splitting dataset into train and test split
training_size=int(len(df1)*0.65)
test_size=len(df1)-training_size
train_data,test_data=df1[0:training_size,:],df1[training_size:len(df1),:1]

In [16]:
training_size,test_size

(81534, 43904)

In [17]:
train_data

array([[0.        ],
       [0.08614141],
       [0.01162729],
       ...,
       [0.48484432],
       [0.35211968],
       [0.48241094]])

In [18]:
test_data

array([[0.35186576],
       [0.47718448],
       [0.3470625 ],
       ...,
       [0.16571271],
       [0.08788709],
       [0.16571271]])

### Convert an array of values(numpy array) into a dataset Matrix

In [19]:
import numpy
def create_dataset(dataset, time_step=1):
    dataX, dataY = [], []
    for i in range(len(dataset)-time_step-1):
        a = dataset[i:(i+time_step), 0]   
        dataX.append(a)
        dataY.append(dataset[i + time_step, 0])
    return numpy.array(dataX), numpy.array(dataY)

In [22]:
# reshape 
time_step = 10
X_train, y_train = create_dataset(train_data, time_step)
X_test, ytest = create_dataset(test_data, time_step)

In [23]:
print(X_train.shape), print(y_train.shape)

(81523, 10)
(81523,)


(None, None)

In [24]:
y_train

array([0.02114919, 0.14533586, 0.01936119, ..., 0.34454448, 0.48484432,
       0.35211968])

In [25]:
X_train

array([[0.        , 0.08614141, 0.01162729, ..., 0.13436452, 0.01732985,
        0.13982374],
       [0.08614141, 0.01162729, 0.08783419, ..., 0.01732985, 0.13982374,
        0.02114919],
       [0.01162729, 0.08783419, 0.01158497, ..., 0.13982374, 0.02114919,
        0.14533586],
       ...,
       [0.34526392, 0.44710587, 0.35188692, ..., 0.49860875, 0.3555264 ,
        0.48483374],
       [0.44710587, 0.35188692, 0.45610935, ..., 0.3555264 , 0.48483374,
        0.34454448],
       [0.35188692, 0.45610935, 0.35294491, ..., 0.48483374, 0.34454448,
        0.48484432]])

In [26]:
print(X_test.shape), print(ytest.shape)

(43893, 10)
(43893,)


(None, None)

### Reshaping input to be [samples ,time steps, features] from [samples,features]

In [27]:
# reshape input to be [samples, time steps, features] which is required for LSTM
X_train =X_train.reshape(X_train.shape[0],X_train.shape[1] , 1)
X_test = X_test.reshape(X_test.shape[0],X_test.shape[1] , 1)

## Building the Model

In [28]:
model=Sequential()
model.add(LSTM(4,return_sequences=True,input_shape=(10,1)))
model.add(LSTM(4,return_sequences=True))
model.add(LSTM(4))
model.add(Dense(1))
model.compile(loss='mean_squared_error',optimizer='adam')


In [29]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 10, 4)             96        
_________________________________________________________________
lstm_1 (LSTM)                (None, 10, 4)             144       
_________________________________________________________________
lstm_2 (LSTM)                (None, 4)                 144       
_________________________________________________________________
dense (Dense)                (None, 1)                 5         
Total params: 389
Trainable params: 389
Non-trainable params: 0
_________________________________________________________________


In [37]:
model.fit(X_train,y_train,validation_data=(X_test,ytest),epochs=100,batch_size=64,verbose=1)

Epoch 1/100
1274/1274 [==============================] - 15s 12ms/step - loss: 2.5775e-04 - val_loss: 2.9515e-04
Epoch 2/100
1274/1274 [==============================] - 15s 12ms/step - loss: 2.0832e-04 - val_loss: 2.0048e-04
Epoch 3/100
1274/1274 [==============================] - 15s 12ms/step - loss: 1.8868e-04 - val_loss: 1.7239e-04
Epoch 4/100
1274/1274 [==============================] - 15s 12ms/step - loss: 1.6867e-04 - val_loss: 1.5317e-04
Epoch 5/100
1274/1274 [==============================] - 15s 12ms/step - loss: 1.5406e-04 - val_loss: 2.9360e-04
Epoch 6/100
1274/1274 [==============================] - 15s 12ms/step - loss: 1.4398e-04 - val_loss: 1.3563e-04
Epoch 7/100
1274/1274 [==============================] - 15s 12ms/step - loss: 1.3899e-04 - val_loss: 1.3177e-04
Epoch 8/100
1274/1274 [==============================] - 15s 12ms/step - loss: 1.2835e-04 - val_loss: 1.4718e-04
Epoch 9/100
1274/1274 [==============================] - 15s 12ms/step - loss: 1.2341e-04 - val_

### Predicting the data

In [70]:
### Lets Do the prediction and check performance metrics
train_predict=model.predict(X_train)
test_predict=model.predict(X_test)

### Transform to original form By Inversing as we had scaled earlier

In [71]:
X_train

array([[[0.        ],
        [0.08614141],
        [0.01162729],
        ...,
        [0.13436452],
        [0.01732985],
        [0.13982374]],

       [[0.08614141],
        [0.01162729],
        [0.08783419],
        ...,
        [0.01732985],
        [0.13982374],
        [0.02114919]],

       [[0.01162729],
        [0.08783419],
        [0.01158497],
        ...,
        [0.13982374],
        [0.02114919],
        [0.14533586]],

       ...,

       [[0.34526392],
        [0.44710587],
        [0.35188692],
        ...,
        [0.49860875],
        [0.3555264 ],
        [0.48483374]],

       [[0.44710587],
        [0.35188692],
        [0.45610935],
        ...,
        [0.3555264 ],
        [0.48483374],
        [0.34454448]],

       [[0.35188692],
        [0.45610935],
        [0.35294491],
        ...,
        [0.48483374],
        [0.34454448],
        [0.48484432]]])

In [72]:
train_predict

array([[-0.00049279],
       [-0.00052647],
       [-0.00056046],
       ...,
       [-0.00185698],
       [-0.00184162],
       [-0.00184648]], dtype=float32)

In [73]:
##Transformback to original form
train_predict=scaler.inverse_transform(train_predict)
test_predict=scaler.inverse_transform(test_predict)

In [74]:

train_predict

array([[2236.9343],
       [2236.9023],
       [2236.87  ],
       ...,
       [2235.6448],
       [2235.6594],
       [2235.6548]], dtype=float32)

In [75]:
y_train

array([0.02114919, 0.14533586, 0.01936119, ..., 0.34454448, 0.48484432,
       0.35211968])

#### Calculate the RMSE performance matrix

In [76]:
### Calculate RMSE performance metrics
import math
from sklearn.metrics import mean_squared_error
math.sqrt(mean_squared_error(y_train,train_predict))

2235.4352729206043

In [77]:
### Test Data RMSE
math.sqrt(mean_squared_error(ytest,test_predict))

2235.5317132554633

In [78]:
len(test_data)

43904

### Predict the output for future 30 days

In [79]:
x_input=test_data[43804:].reshape(1,-1)
x_input.shape


(1, 100)

In [80]:
temp_input=list(x_input)
temp_input=temp_input[0].tolist()

In [81]:
temp_input

[0.09161121044446041,
 0.17756218326474027,
 0.09162179032787021,
 0.17756218326474027,
 0.09459473756599168,
 0.17756218326474027,
 0.09319819295591314,
 0.17747754419746276,
 0.09462647721622108,
 0.1705159809138901,
 0.09462647721622108,
 0.17050540103048029,
 0.09318761307250378,
 0.17050540103048029,
 0.09438313989779834,
 0.17050540103048029,
 0.0943937197812077,
 0.17050540103048029,
 0.09428792094711103,
 0.17050540103048029,
 0.09162179032787021,
 0.17050540103048029,
 0.09426676118029143,
 0.17050540103048029,
 0.09008770723346604,
 0.17050540103048029,
 0.09007712735005624,
 0.17050540103048029,
 0.0852950200488789,
 0.17790073953385033,
 0.08270294861350624,
 0.16145960071520005,
 0.08208931537574404,
 0.16145960071520005,
 0.07928564627217805,
 0.17299167363175627,
 0.07963478242469746,
 0.16250700917275873,
 0.08450152879315231,
 0.16250700917275873,
 0.08600387223732753,
 0.16250700917275873,
 0.08733693754694771,
 0.162834985558459,
 0.08628952908938903,
 0.162845565441

In [82]:
# demonstrate prediction for next 30 days
from numpy import array

lst_output=[]
n_steps=100
i=0
while(i<30):
    
    if(len(temp_input)>100):
        #print(temp_input)
        x_input=np.array(temp_input[1:])
        print("{} day input {}".format(i,x_input))
        x_input=x_input.reshape(1,-1)
        x_input = x_input.reshape((1, n_steps, 1))
        #print(x_input)
        yhat = model.predict_classes(x_input, verbose=0)
        print("{} day output {}".format(i,yhat))
        temp_input.extend(yhat[0].tolist())
        temp_input=temp_input[1:]
        #print(temp_input)
        lst_output.extend(yhat.tolist())
        i=i+1
    else:
        x_input = x_input.reshape((1, n_steps,1))
        yhat = model.predict_classes(x_input, verbose=0)
        print(yhat[0])
        temp_input.extend(yhat[0].tolist())
        print(len(temp_input))
        lst_output.extend(yhat.tolist())
        i=i+1
    

print(lst_output)

[0]
101
1 day input [0.17756218 0.09162179 0.17756218 0.09459474 0.17756218 0.09319819
 0.17747754 0.09462648 0.17051598 0.09462648 0.1705054  0.09318761
 0.1705054  0.09438314 0.1705054  0.09439372 0.1705054  0.09428792
 0.1705054  0.09162179 0.1705054  0.09426676 0.1705054  0.09008771
 0.1705054  0.09007713 0.1705054  0.08529502 0.17790074 0.08270295
 0.1614596  0.08208932 0.1614596  0.07928565 0.17299167 0.07963478
 0.16250701 0.08450153 0.16250701 0.08600387 0.16250701 0.08733694
 0.16283499 0.08628953 0.16284557 0.08626837 0.16286673 0.0801003
 0.16793449 0.08610967 0.16285615 0.08610967 0.16285615 0.08610967
 0.16285615 0.08316846 0.16286673 0.08317904 0.16285615 0.08633185
 0.17301283 0.08633185 0.16289846 0.08633185 0.17638782 0.08528444
 0.17638782 0.08583459 0.16353326 0.08583459 0.17366879 0.08528444
 0.1686116  0.0853056  0.16288788 0.0853056  0.17638782 0.0853056
 0.17638782 0.08604619 0.16285615 0.08738984 0.16283499 0.09007713
 0.16283499 0.09007713 0.16283499 0.09007713

In [83]:
lst_output=scaler.inverse_transform(lst_output)

Price after 30 days

In [84]:
lst_output[-1]

array([2237.4])